# Sprint 1: Formalización del problema

## Espacio de codificación de soluciones


## Codificación de soluciones


In [ ]:
### Propón algunas soluciones al problema como ejemplo
sol_1=[]
sol_2=[]

### Función objetivo


In [ ]:
# pip install ipynb
from itertools import product
from ???? import crear_grafo
import community
import networkx as nx
import numpy as np

### Función objetivo
def modularidad(G, particion, weight='weight'):
    
    ### TODO 15-20 lineas
    
    return fitness

### llamadas generales
G = crear_grafo()

## nuestra implementación
partition1 = dict(zip(G.nodes, sol_1))
partition2 = dict(zip(G.nodes, sol_2))

print("Fitness solucion 1: {} ".format(modularidad(G, partition1)))
print("Fitness solucion 2: {} ".format(modularidad(G, partition2)))